# Applied Data Science Capstone Project
## Week 3 assignment
##### Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

# Part A

## Create a dataframe of neighborhood data for Toronto, Canada: postal codes, borough names, and neighborhood names

## Import needed libraries

In [673]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

print("Imports done")

Imports done


## Get the neighborhood data from Wikipedia, which has a table of the boroughs, neighborhoods, and postal codes of Toronto


In [674]:
response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

print("The response data has been retrieved")

The response data has been retrieved


## Use BeautifulSoup for parsing the response data.
##### Working assumption is that the table we want is the first table in the response.

In [675]:
soup = BeautifulSoup(response.content, 'lxml')
table = soup.find('table')

## Convert the HTML table to a Pandas DataFrame

In [676]:
df_tor = pd.read_html(str(table), header=0)[0]
df_tor.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


## Clean the data per the assignment specification

* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

## Set the column names to 'PostalCode', 'Borough', and 'Neighborhood

In [677]:
column_headers = ['PostalCode', 'Borough', 'Neighborhood']
df_tor.columns = column_headers
df_tor.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Only process the cells that have an assigned borough. Ignore cells with a borough that is 'Not assigned'.

In [678]:
# Remove rows where 'Borough' is 'Not assigned'. Don't forget to re-index
# the rows!
df_tor = df_tor[df_tor['Borough'] != 'Not assigned'].reset_index(drop=True)
df_tor.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


## If a cell has a borough but a 'Not assigned' neighborhood, then change the neighborhood to the name of its borough.

In [679]:
df_tor['Neighborhood'].loc[df_tor['Neighborhood'] == 'Not assigned'] = df_tor['Borough']
df_tor.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


## Combine neighborhoods with the same postal code into a single row, with the neighborhood names separated by a comma and a space, e.g., 'Neighborhood1, Neighborhood2'.

In [680]:
# I sort by the values first, so the joined neighborhoods appear in alphabetic order.
df_tor = df_tor.sort_values(['PostalCode', 'Neighborhood']).groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

## Display the shape of the dataframe

In [681]:
print('Shape of df_tor is {}.'.format(df_tor.shape))

Shape of df_tor is (103, 3).


# Part B

## Add each neighborhood's geographical coordinates, latitude and longitude, to the dataframe

##### I was unable to make geocoder work. I found that the google provider would not return any results; the osm provider returned unreliable results; and other providers required creating an account or paying for the access. Therefore , I am using the CSV file that was provided with the project instructions. The CSV file is included on GitHub with this notebook.

## Load the CSV file that includes latitude and longitude for the Toronto postal codes.

In [682]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
print('Shape of coordinates is {}.'.format(coordinates.shape))
coordinates.head()

Shape of coordinates is (103, 3).


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge the coordinates dataframe into df_tor

In [683]:
df_tor = df_tor.merge(coordinates, left_on='PostalCode', right_on='Postal Code', how='left')

# The 'Postal Code' column was included in the merged data, but it is not needed.
df_tor.drop('Postal Code', axis=1, inplace=True)

df_tor.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Remove any rows if there are null values for Latitude or Longitude.
##### For this project, I know that the input data matches correctly, but in general, if the geospatial coordinates are not available, then we likely cannot use the data.

In [684]:
print('df_tor shape before dropping rows with NaN coordinates: {}'.format(df_tor.shape))
df_tor.dropna(subset = ['Latitude', 'Longitude'], inplace=True)
df_tor.reset_index(drop=True)
print('df_tor shape after dropping rows with NaN coordinates: {}'.format(df_tor.shape))

df_tor shape before dropping rows with NaN coordinates: (103, 5)
df_tor shape after dropping rows with NaN coordinates: (103, 5)


## Examine the first and last rows to make sure the dataframe looks right.

In [685]:
df_tor.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [686]:
df_tor.tail(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
93,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
94,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724
95,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201
96,M9L,North York,Humber Summit,43.756303,-79.565963
97,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
102,M9W,Etobicoke,Northwest,43.706748,-79.594054


# Part C

## Explore and cluster the neighborhoods in Toronto

## Import additional libraries

In [687]:
import numpy as np

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Imports done')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Imports done


## Define some utility functions

In [688]:
# Given an address, return geographical coordinates (latitude, longitude)
def get_geo_coords(address=None):
    if address is None or not isinstance(address, str):
        return None, None

    geo_locator = Nominatim(user_agent="my-homework")
    location = geo_locator.geocode(address)

    if location is not None:
        return location.latitude, location.longitude
    else:
        return None, None

In [689]:
# Given latitude and longitude, return a folium map. Markes are added if a
# Pandas DataFrame with individual neighborhood names, boroughs, latitudes,
# and longitudes, is provided.
def get_map_with_markers(latitude=None, longitude=None, df=None, zoom=10):
    # Cannot make a map without coordinates.
    if latitude is None or longitude is None:
        return None

    folium_map = folium.Map(location=[latitude, longitude], zoom_start=zoom)

    # If a dataframe was provided, add markers
    if df is not None:
        # Add markers to the map
        for lat, lng, borough, neighborhood in zip(df['Latitude'],
                                                   df['Longitude'],
                                                   df['Borough'],
                                                   df['Neighborhood']):
            label = '{}: {}'.format(borough, neighborhood)
            label = folium.Popup(label, parse_html=True)
            folium.CircleMarker([lat, lng],
                                radius=5,
                                popup=label,
                                color='blue',
                                fill=True,
                                fill_color='#3186cc',
                                fill_opacity=0.7,
                                parse_html=False).add_to(folium_map)  

    return folium_map

In [690]:
# Display high-level information about a dataframe that contains 'Borough'
# and 'Neighborhood' columns.
def display_neighborhood_stats(df=None):
    if df is not None:
        print('This dataframe has {} boroughs and {} neighborhoods'.format(len(df['Borough'].unique()),
                                                                   df.shape[0]))
        print()
        print('Borough names and number of neighborhoods per borough:')
        print('------------------------------------------------------')
        print(df['Borough'].value_counts())

In [691]:
# Given a dataframe that contains 'Latitude' and 'Longitude' columns,
# return the average latitude and longitude.
def get_average_latitude_and_longitude(df=None):
    if df is None or 'Latitude' not in df.columns or 'Longitude' not in df.columns:
        return None, None

    lat_long_list = list(df[['Latitude', 'Longitude']].itertuples(index=False, name=None))
    lat_long_avg = list(map(lambda x: sum(x) / float(len(x)), zip(*lat_long_list)))
    return lat_long_avg[0], lat_long_avg[1]

In [692]:
# Return the category of a venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [693]:
# Given neighborhood names and geographical coordinates, return a
# Pandas dataframe of nearby venues
def get_nearby_venues(neighborhoods, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(neighborhoods, latitudes, longitudes):

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

    return(nearby_venues)

In [694]:
# Return venues sorted in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Some high-level information about the neighborhoods in the dataframe

In [695]:
display_neighborhood_stats(df_tor)

This dataframe has 11 boroughs and 103 neighborhoods

Borough names and number of neighborhoods per borough:
------------------------------------------------------
North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64


## Let's take a look at the overall neighborhood distribution in Toronto

In [696]:
tor_latitude, tor_longitude = get_geo_coords('Toronto, ON, Canada')
print('The geograpical coordinate of Toronto are {}, {}.'.format(tor_latitude, tor_longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Create a map of Toronto, with markers for the neighborhoods

In [720]:
map_tor = get_map_with_markers(latitude=tor_latitude, longitude=tor_longitude,
                               df=df_tor)
map_tor

## Now let's expolore a subset of the neighborhoods, the boroughs that have 'York' in their name

In [698]:
df_yorks = df_tor[df_tor['Borough'].str.contains('York', regex=False)].reset_index(drop=True)

In [699]:
display_neighborhood_stats(df_yorks)

This dataframe has 3 boroughs and 34 neighborhoods

Borough names and number of neighborhoods per borough:
------------------------------------------------------
North York    24
East York      5
York           5
Name: Borough, dtype: int64


## As we did above for Toronto, create a map of the 'Yorks' that includes neighborhood markers, but this map also include markers for the boroughs

In [719]:
# Find coordinates for each of the Yorks' neighborhoods, and get their
# average to identify a central latitude and longitude.
yorks_latitude, yorks_longitude = get_average_latitude_and_longitude(df_yorks)

print('The average geograpical coordinates of the Yorks neighborhoods are {}, {}.'.format(yorks_latitude,
                                                                                          yorks_longitude))
# Draw a map of the Yorks neighborhoods centered on the average
# latitude and longitude, with markers for the neighborhoods.
map_yorks = get_map_with_markers(latitude=yorks_latitude, longitude=yorks_longitude,
                                 df=df_yorks, zoom=11)

# Add green markers to the map centered on the average latitude and
# longitude of the boroughs.
for borough in df_yorks['Borough'].unique():
    df_borough = df_yorks[df_yorks['Borough'] == borough].reset_index(drop=True)
    borough_latitude, borough_longitude = get_average_latitude_and_longitude(df_borough)
    print('The average geograpical coordinates of the borough {} are {}, {}.'.format(borough,
                                                                                     borough_latitude,
                                                                                     borough_longitude))
    label = 'Borough of {}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([borough_latitude, borough_longitude],
                       radius=10,
                       popup=label,
                       color='green',
                       fill=True,
                       fill_color='#007f00',
                       fill_opacity=0.7,
                       parse_html=False).add_to(map_yorks)

map_yorks

The average geograpical coordinates of the Yorks neighborhoods are 43.73284565454544, -79.42236514545455.
The average geograpical coordinates of the borough North York are 43.749061095652166, -79.43024465217391.
The average geograpical coordinates of the borough East York are 43.70030348, -79.33585115999999.
The average geograpical coordinates of the borough York are 43.6907968, -79.47263340000002.


## Set up Foursquare credentials and version

In [722]:
# I removed my personal credentials before submitting
CLIENT_ID = 'YOUR-CLIENT-ID' # your Foursquare ID
CLIENT_SECRET = 'YOUR-CLIENT-SECRET' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

## We will start by examine one neighborhood, Hillcrest Village in the borough of North York

## Get the neighborhood's latitude and longitude

In [702]:
# Get the row number for Hillcrest Village
hillcrest_index = df_yorks[df_yorks['Neighborhood'] == 'Hillcrest Village'].index[0]
print('dataframe index for Hillcrest Village: {}'.format(hillcrest_index))
hillcrest_latitude = df_yorks.loc[hillcrest_index, 'Latitude']
hillcrest_longitude = df_yorks.loc[hillcrest_index, 'Longitude']
print('Latitude and longitude values of Hillcrest Village are {}, {}.'.format(hillcrest_latitude,
                                                                             hillcrest_longitude))

dataframe index for Hillcrest Village: 0
Latitude and longitude values of Hillcrest Village are 43.8037622, -79.3634517.


## Get the top 100 venues that are in Hillcrest Village within a radius of 1000 meters (Foursquare might return less than 100)

In [703]:
radius = 1000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, hillcrest_latitude, hillcrest_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bd9785d9fb6b71cf91401e7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 20,
  'suggestedBounds': {'ne': {'lat': 43.81276220900001,
    'lng': -79.35100467075661},
   'sw': {'lat': 43.79476219099999, 'lng': -79.37589872924339}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd9842be914a593adbd56fa',
       'name': 'Tastee',
       'location': {'address': '3913 Don Mills Rd.',
        'crossStreet': 'at Cliffwood Rd.',
        'lat': 43.80772211146167,
        'lng': -79.35679781099806,
        'labeledLatLngs': [{'label': 'display',
      

## Clean the json data, then use it to build a Pandas dataframe

In [704]:
venues = results['response']['groups'][0]['items']

# "Flatten" the JSON data into a table.
nearby_venues = json_normalize(venues) # flatten JSON

# Eliminate the columns we do not care about.
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# The 'venue.categories' column contains a diction of a number of values,
# including the category name. Extract just the category name.
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# Rename the columns to just the part of the column name to the right of
# the right-most '.', e.g., rename 'venue.location.lng' to 'lng'.
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Tastee,Bakery,43.807722,-79.356798
1,고려삼계탕 Korean Ginseng Chicken Soup & Bibimbap,Korean Restaurant,43.798391,-79.369187
2,Galati,Grocery Store,43.797831,-79.369410
3,Cummer Park,Park,43.799564,-79.371175
4,Tim Hortons,Coffee Shop,43.798945,-79.369644


## How many venues in Hillcrest Village were returned by Foursquare?

In [705]:
print('There were {} venues returned by Foursquare.'.format(nearby_venues.shape[0]))

There were 20 venues returned by Foursquare.


## Now let's expand our exploration to venues in all the York neighborhoods

In [706]:
# Note: Using ".tolist()" might be extraneous
yorks_neighbors = df_yorks['Neighborhood'].tolist()
yorks_latitudes = df_yorks['Latitude'].tolist()
yorks_longitudes = df_yorks['Longitude'].tolist()

yorks_venues = get_nearby_venues(yorks_neighbors,
                                 yorks_latitudes,
                                 yorks_longitudes)
yorks_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,Hillcrest Village,43.803762,-79.363452,A.Y. Jackson Secondary School Track,43.805068,-79.366677,Athletics & Sports
5,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,CF Fairview Mall,43.777897,-79.344727,Shopping Mall
6,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store
7,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Purdys Chocolatier,43.778099,-79.343853,Candy Store
8,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Apple Fairview,43.777952,-79.343582,Electronics Store
9,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Michel's Baguette,43.777082,-79.344557,Bakery


## A little more cleaning of the df_yorks dataframe: It is possible that a neighborhood might not have any venues, so remove any such neighborhoods from the dataframe

In [707]:
# This is necessary, or else the clustering activity later on in
# this notebook will produce errors when trying to add the cluster
# labels column to the df_yorks dataframe.

# One potential issue is if there is more than one neighborhood
# with the same name, but hopefully that is rare or unlikely.

# Take the difference between two sets to see if there are any
# neighborhoods without venues
neighborhoods_with_venues = set(yorks_venues['Neighborhood'].unique().tolist())
venueless_neighborhoods = list(set(yorks_neighbors) - neighborhoods_with_venues)

if len(venueless_neighborhoods) == 0:
    print('All neighborhoods have at least one venue.')
else:
    print('Neighborhoods without venues: {}'.format(venueless_neighborhoods))
    # Remove the rows without venues.
    df_yorks = df_yorks[~df_yorks['Neighborhood'].isin(venueless_neighborhoods)].reset_index()

Neighborhoods without venues: ['Newtonbrook, Willowdale']


## Examine the shape and first few records of the venues dataframe

In [708]:
print('yorks_venues shape: {}'.format(yorks_venues.shape))
yorks_venues.head(10)

yorks_venues shape: (325, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,Hillcrest Village,43.803762,-79.363452,A.Y. Jackson Secondary School Track,43.805068,-79.366677,Athletics & Sports
5,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,CF Fairview Mall,43.777897,-79.344727,Shopping Mall
6,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store
7,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Purdys Chocolatier,43.778099,-79.343853,Candy Store
8,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Apple Fairview,43.777952,-79.343582,Electronics Store
9,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Michel's Baguette,43.777082,-79.344557,Bakery


## How many venues were returned for each neighborhood?

In [709]:
yorks_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
"CFB Toronto, Downsview East",3,3,3,3,3,3
Caledonia-Fairbanks,6,6,6,6,6,6
"Del Ray, Keelsdale, Mount Dennis, Silverthorn",7,7,7,7,7,7
Don Mills North,5,5,5,5,5,5
"Don Mills South, Flemingdon Park",21,21,21,21,21,21
Downsview Central,3,3,3,3,3,3


## Examine each neighborhood

### Create a "onehot" table of the neighborhoods and their venues

In [710]:
# one hot encoding
yorks_onehot = pd.get_dummies(yorks_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
yorks_onehot['Neighborhood'] = yorks_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [yorks_onehot.columns[-1]] + list(yorks_onehot.columns[:-1])
yorks_onehot = yorks_onehot[fixed_columns]

print('yorks_onehot shape: {}'.format(yorks_onehot.shape))
print(yorks_onehot.shape)
yorks_onehot.head(10)

yorks_onehot shape: (325, 127)
(325, 127)


,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,...,Toy / Game Store,Trail,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Now group rows by neighborhood, and take the mean of the frequency of occurrence of each category

In [711]:
# After the groupyby, the index becomes Neighborhood, so reset_index()
# moves the Neighborhood back to a regular column, and the DataFrame is
# 0..nnn indexed.
yorks_grouped = yorks_onehot.groupby('Neighborhood').mean().reset_index()
print('Shape of yorks_grouped is {}.'.format(yorks_grouped.shape))
yorks_grouped

Shape of yorks_grouped is (33, 127).


,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,...,Toy / Game Store,Trail,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.055556,...,0.000000,0.00,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.250000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CFB Toronto, Downsview East",0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Caledonia-Fairbanks,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000
5,"Del Ray, Keelsdale, Mount Dennis, Silverthorn",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.00,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Don Mills North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Don Mills South, Flemingdon Park",0.000000,0.000000,0.000000,0.000000,0.095238,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Downsview Central,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Downsview Northwest,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Display each neighborhood and its five top venues

In [712]:
num_top_venues = 5

for hood in yorks_grouped['Neighborhood']:
    print("----"+hood+"----")
    # pandas.DataFrame.T is an "accessor" to the transpose() method
    temp = yorks_grouped[yorks_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
              venue  freq
0       Coffee Shop  0.11
1       Bridal Shop  0.06
2     Deli / Bodega  0.06
3  Sushi Restaurant  0.06
4        Restaurant  0.06


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1   Chinese Restaurant  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4        Movie Theater  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.08
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.08
3         Grocery Store  0.04
4               Butcher  0.04


----CFB Toronto, Downsview East----
         venue  freq
0      Airport  0.33
1         Park  0.33
2     Bus Stop  0.33
3         Pool  0.00
4  Pizza Place  0.00


----Caledonia-Fairbanks----
                  venue  freq
0                  Park  0.33
1  Fast Food Restaurant  0.17
2         Women's Store  0.17
3              Pharmacy  0.17
4                Mar

## Create a new dataframe, then display the 10 most commone venues for each neighborhood

In [713]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
yorks_venues_sorted = pd.DataFrame(columns=columns)
yorks_venues_sorted['Neighborhood'] = yorks_grouped['Neighborhood']

for ind in np.arange(yorks_grouped.shape[0]):
    yorks_venues_sorted.iloc[ind, 1:] = return_most_common_venues(yorks_grouped.iloc[ind, :], num_top_venues)

yorks_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Pharmacy,Supermarket,Fried Chicken Joint,Ice Cream Shop,Diner,Deli / Bodega,Pet Store,Pizza Place,Restaurant
1,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Café,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
2,"Bedford Park, Lawrence Manor East",Fast Food Restaurant,Italian Restaurant,Coffee Shop,Greek Restaurant,Comfort Food Restaurant,Pub,Butcher,Pizza Place,Pharmacy,Juice Bar
3,"CFB Toronto, Downsview East",Airport,Park,Bus Stop,Yoga Studio,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega
4,Caledonia-Fairbanks,Park,Pharmacy,Women's Store,Fast Food Restaurant,Market,Department Store,Diner,Dim Sum Restaurant,Dessert Shop,Curling Ice
5,"Del Ray, Keelsdale, Mount Dennis, Silverthorn",Fast Food Restaurant,Restaurant,Coffee Shop,Fried Chicken Joint,Turkish Restaurant,Sandwich Place,Deli / Bodega,Dim Sum Restaurant,Dessert Shop,Department Store
6,Don Mills North,Caribbean Restaurant,Gym / Fitness Center,Japanese Restaurant,Café,Basketball Court,Yoga Studio,Dim Sum Restaurant,Dog Run,Discount Store,Diner
7,"Don Mills South, Flemingdon Park",Beer Store,Coffee Shop,Asian Restaurant,Gym,General Entertainment,Japanese Restaurant,Italian Restaurant,Restaurant,Sporting Goods Shop,Fast Food Restaurant
8,Downsview Central,Food Truck,Home Service,Baseball Field,Yoga Studio,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
9,Downsview Northwest,Liquor Store,Grocery Store,Gym / Fitness Center,Athletics & Sports,Department Store,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


## Cluster the neighborhoods

#### Run <i>k</i>-means to cluster the neighborhoods into five clusters

In [714]:
kclusters = 5

yorks_grouped_clustering = yorks_grouped.drop('Neighborhood', 1)

# run k-means clustering
#kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(yorks_grouped_clustering)

# k_means = KMeans(init = "k-means++", n_clusters = 4, n_init = 12)
kmeans = KMeans(n_clusters=kclusters, init="k-means++", n_init=12, random_state=0).fit(yorks_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ #[0:10]

array([4, 4, 4, 1, 1, 4, 4, 4, 0, 4, 1, 1, 0, 4, 1, 4, 2, 1, 4, 4, 1, 4,
       4, 1, 4, 3, 4, 4, 1, 4, 4, 1, 1], dtype=int32)

## Create a dataframe that includes the cluster and 10 most common venues for each neighborhood

In [715]:
yorks_merged = df_yorks.copy()

# Add clustering labels.
yorks_merged['Cluster Labels'] = kmeans.labels_

# Nor merge the Yorks venues with the Yorks neighborhood information (yords_merged)
yorks_merged = yorks_merged.join(yorks_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

yorks_merged.head() # check the last columns!

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M2H,North York,Hillcrest Village,43.803762,-79.363452,4,Golf Course,Athletics & Sports,Mediterranean Restaurant,Dog Run,Pool,Construction & Landscaping,Convenience Store,Comfort Food Restaurant,Cosmetics Shop,Electronics Store
1,1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4,Clothing Store,Fast Food Restaurant,Coffee Shop,Bakery,Women's Store,Asian Restaurant,Toy / Game Store,Cosmetics Shop,Candy Store,Boutique
2,2,M2K,North York,Bayview Village,43.786947,-79.385975,4,Chinese Restaurant,Bank,Japanese Restaurant,Café,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
3,3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,1,Cafeteria,Yoga Studio,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store
4,5,M2N,North York,Willowdale South,43.770120,-79.408493,1,Restaurant,Ramen Restaurant,Pizza Place,Coffee Shop,Café,Sandwich Place,Plaza,Bubble Tea Shop,Indonesian Restaurant,Japanese Restaurant


## Create a visualization of the resulting clusters

In [718]:
map_clusters = folium.Map(location=[yorks_latitude, yorks_longitude], zoom_start=10)

# Set the color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(yorks_merged['Latitude'],
                                  yorks_merged['Longitude'],
                                  yorks_merged['Neighborhood'],
                                  yorks_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Now let's look at the individual clusters

In [717]:
# Display the data for each cluster
for cluster in range(kclusters):
    print('*' * 12)
    print(' Cluster {}'.format(cluster))
    print('*' * 12)
    display(yorks_merged.loc[yorks_merged['Cluster Labels'] == cluster,
                             yorks_merged.columns[[3] + list(range(7, yorks_merged.shape[1]))]])
    print()

************
 Cluster 0
************


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Don Mills North,Caribbean Restaurant,Gym / Fitness Center,Japanese Restaurant,Café,Basketball Court,Yoga Studio,Dim Sum Restaurant,Dog Run,Discount Store,Diner
12,"CFB Toronto, Downsview East",Airport,Park,Bus Stop,Yoga Studio,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega



************
 Cluster 1
************


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Silver Hills, York Mills",Cafeteria,Yoga Studio,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store
4,Willowdale South,Restaurant,Ramen Restaurant,Pizza Place,Coffee Shop,Café,Sandwich Place,Plaza,Bubble Tea Shop,Indonesian Restaurant,Japanese Restaurant
10,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Pharmacy,Supermarket,Fried Chicken Joint,Ice Cream Shop,Diner,Deli / Bodega,Pet Store,Pizza Place,Restaurant
11,"Northwood Park, York University",Metro Station,Bar,Massage Studio,Coffee Shop,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store,Electronics Store,Dessert Shop
14,Downsview Central,Food Truck,Home Service,Baseball Field,Yoga Studio,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
17,"Parkview Hill, Woodbine Gardens",Pizza Place,Fast Food Restaurant,Bank,Gym / Fitness Center,Intersection,Pet Store,Pharmacy,Rock Climbing Spot,Breakfast Spot,Gastropub
20,Thorncliffe Park,Indian Restaurant,Sandwich Place,Yoga Studio,Burger Joint,Grocery Store,Gym,Coffee Shop,Park,Pharmacy,Bus Station
23,"Lawrence Heights, Lawrence Manor",Clothing Store,Accessories Store,Fraternity House,Miscellaneous Shop,Shoe Store,Event Space,Boutique,Coffee Shop,Furniture / Home Store,Arts & Crafts Store
28,"Del Ray, Keelsdale, Mount Dennis, Silverthorn",Fast Food Restaurant,Restaurant,Coffee Shop,Fried Chicken Joint,Turkish Restaurant,Sandwich Place,Deli / Bodega,Dim Sum Restaurant,Dessert Shop,Department Store
31,"Emery, Humberlea",Furniture / Home Store,Baseball Field,Yoga Studio,Department Store,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Deli / Bodega



************
 Cluster 2
************


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Victoria Village,Coffee Shop,Portuguese Restaurant,Pizza Place,Hockey Arena,Intersection,Yoga Studio,Department Store,Diner,Dim Sum Restaurant,Dessert Shop



************
 Cluster 3
************


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Humewood-Cedarvale,Trail,Park,Field,Hockey Arena,Deli / Bodega,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Yoga Studio



************
 Cluster 4
************


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hillcrest Village,Golf Course,Athletics & Sports,Mediterranean Restaurant,Dog Run,Pool,Construction & Landscaping,Convenience Store,Comfort Food Restaurant,Cosmetics Shop,Electronics Store
1,"Fairview, Henry Farm, Oriole",Clothing Store,Fast Food Restaurant,Coffee Shop,Bakery,Women's Store,Asian Restaurant,Toy / Game Store,Cosmetics Shop,Candy Store,Boutique
2,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Café,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
5,York Mills West,Park,Bank,Yoga Studio,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega
6,Willowdale West,Pharmacy,Coffee Shop,Butcher,Pizza Place,Grocery Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Dog Run,Curling Ice
7,Parkwoods,Park,Food & Drink Shop,Fast Food Restaurant,Yoga Studio,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice
9,"Don Mills South, Flemingdon Park",Beer Store,Coffee Shop,Asian Restaurant,Gym,General Entertainment,Japanese Restaurant,Italian Restaurant,Restaurant,Sporting Goods Shop,Fast Food Restaurant
13,Downsview West,Grocery Store,Park,Shopping Mall,Bank,Yoga Studio,Deli / Bodega,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
15,Downsview Northwest,Liquor Store,Grocery Store,Gym / Fitness Center,Athletics & Sports,Department Store,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
18,Woodbine Heights,Skating Rink,Park,Asian Restaurant,Cosmetics Shop,Curling Ice,Beer Store,Yoga Studio,Dim Sum Restaurant,Discount Store,Diner
